In [58]:
# Importar paqueteria necesaria
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
import os

In [ ]:
# Especificar ruta al repo clonado
os.chdir("/home/cesar/Documentos/Banco_Base")

# Lectura de la informacion
data_agosto = pd.read_csv('./Ejercicio_1/datasets/2024-08.csv')
data_sept = pd.read_csv('./Ejercicio_1/datasets/2024-09.csv')
data_oct = pd.read_csv('./Ejercicio_1/datasets/2024-10.csv')

data = pd.concat([data_agosto, data_sept, data_oct],ignore_index=True)
data.head(10)
 

In [ ]:
# Porcentaje de faltantes por columna
data['Genero_Usuario'] = data['Genero_Usuario'].replace('?', np.nan)
data.isna().mean() * 100


In [ ]:
################## Inciso 1: Analisis exploratorio
# Numero de estaciones
print(pd.concat([data['Ciclo_Estacion_Retiro'], data['Ciclo_EstacionArribo']]).nunique())
# Histograma del genero de usuarios
frecuencias = data['Genero_Usuario'].value_counts(normalize=True).reset_index()
frecuencias.columns = ['Genero_Usuario', 'frecuencia']
plt.figure(figsize=(8, 6))
sns.barplot(x='Genero_Usuario', y='frecuencia', data=frecuencias, palette='pastel', edgecolor='black')
plt.title('Usuarios por genero')
plt.xlabel('Genero')
plt.ylabel('Frecuencia')
plt.ylim(0, 1)
plt.savefig('./Ejercicio_1/figures/genero_usuarios.png')
plt.show()

In [ ]:
# Cantidad de bicicletas usadas
print(data['Bici'].nunique())
# Cantidad de viajes por edad
plt.figure(figsize=(8, 6))
sns.histplot(data['Edad_Usuario'], bins=10, kde=True, color='blue')
plt.title('Edades')
plt.xlabel('Rangos de edad')
plt.ylabel('Cantidad de viajes')
plt.savefig('./Ejercicio_1/figures/edad_usuarios.png')
plt.show()
# Edad promedio por genero
print(data.groupby(['Genero_Usuario']).agg('Edad_Usuario').mean())

In [ ]:
# Calculo de duracion de viajes en minutos
data['Hora_Retiro'] = pd.to_datetime(data['Hora_Retiro'], format='%H:%M:%S')
data['Hora_Arribo'] = pd.to_datetime(data['Hora_Arribo'], format='%H:%M:%S')
data['duracion_viaje'] = (
    (data['Hora_Arribo'] - data['Hora_Retiro'])
    .dt.total_seconds()
    .div(60)    # Tiempo en minutos
    .mod(1440)  # Manejar diferencias de días
)
data[['Hora_Retiro','Hora_Arribo','duracion_viaje']].head(10)


In [ ]:
# Histograma de duraciones
plt.figure()
figsize=(8, 6)
sns.histplot(data['duracion_viaje'], kde=False, color='blue')
plt.title('Duracion en minutos')
plt.xlabel('Rangos de duracion')
plt.ylabel('Cantidad de viajes')
plt.xlim(0,100)
plt.savefig('./Ejercicio_1/figures/duracion_viajes.png')
plt.show()

In [5]:
############################## Inciso 2: Afluencia y outliers
## Ruta origen-destino
data['viaje_inicio_destino'] = data["Ciclo_Estacion_Retiro"] + '/' + data["Ciclo_EstacionArribo"]
# Conteo de viajes en intervalos de 30min
data['intervalo'] = data['Hora_Retiro'].dt.floor('30min')  # Redondear a la hora más cercana
afluencia = data['intervalo'].value_counts().sort_index()

In [ ]:
## Horarios de mayor afluencia
figsize=(8, 6)
sns.barplot(x=afluencia.index.strftime('%H:%M'), y=afluencia.values, palette='pastel', edgecolor='black')
plt.xlabel('Hora de Inicio')
plt.ylabel('Cantidad de Viajes')
plt.title('Afluencia de Viajes por Intervalo de Tiempo')
plt.xticks(rotation=90)  # Rotar etiquetas si es necesario
plt.savefig('./Ejercicio_1/figures/horarios_afluencia.png')
plt.show()

In [ ]:
# Afluencia por estacion de retiro
estacion = data.groupby(['Ciclo_Estacion_Retiro'])['Ciclo_Estacion_Retiro'].count().reset_index(name='conteo')
estacion = estacion.sort_values(by='conteo', ascending=False)
# Estaciones de afluencia atipica
estacion_q1 = estacion['conteo'].quantile(0.25)
estacion_q3 = estacion['conteo'].quantile(0.75)
iqr = estacion_q3 - estacion_q1
limite_inferior = estacion_q1 - 1.5 * iqr
limite_superior = estacion_q3 + 1.5 * iqr
# Identificar outliers
estacion[(estacion['conteo'] < limite_inferior) | (estacion['conteo'] > limite_superior)]



In [ ]:
# Afluencia por estacion de arribo
estacion_a = data.groupby(['Ciclo_EstacionArribo'])['Ciclo_EstacionArribo'].count().reset_index(name='conteo')
estacion_a = estacion_a.sort_values(by='conteo', ascending=False)
# Estaciones de afluencia atipica
estacion_q1 = estacion_a['conteo'].quantile(0.25)
estacion_q3 = estacion_a['conteo'].quantile(0.75)
iqr = estacion_q3 - estacion_q1
limite_inferior = estacion_q1 - 1.5 * iqr
limite_superior = estacion_q3 + 1.5 * iqr
# Identificar outliers
estacion_a[(estacion_a['conteo'] < limite_inferior) | (estacion_a['conteo'] > limite_superior)]


In [ ]:
# Afluencia por estacion de retiro y horario 
estacion_horario = data.groupby(['Ciclo_Estacion_Retiro','intervalo'])['Ciclo_Estacion_Retiro'].count().reset_index(name='conteo')
estacion_horario = estacion_horario.sort_values(by='conteo', ascending=False)
estacion_horario.head(30)
# ['271-272','266-267','273-274','264-275','237-238','268-269','158-159']

In [ ]:
## Afluencia por ruta
afluencia_ruta = data.groupby(['viaje_inicio_destino'])['viaje_inicio_destino'].count().reset_index(name='conteo')
afluencia_ruta = afluencia_ruta.sort_values(by='conteo', ascending=False)
# Rutas de afluencia atipica
ruta_q1 = afluencia_ruta['conteo'].quantile(0.25)
ruta_q3 = afluencia_ruta['conteo'].quantile(0.75)
iqr = ruta_q3 - ruta_q1
limite_inferior = ruta_q1 - 1.5 * iqr
limite_superior = ruta_q3 + 1.5 * iqr
# Identificar outliers
afluencia_ruta[(afluencia_ruta['conteo'] < limite_inferior) | (afluencia_ruta['conteo'] > limite_superior)]


In [30]:
# Algunas metricas por estacion
edad = data.groupby(['Ciclo_Estacion_Retiro'])['Edad_Usuario'].median().reset_index(name='edad')
edad.columns = ['estacion', 'edad']
duracion = data.groupby(['Ciclo_Estacion_Retiro'])['duracion_viaje'].median().reset_index(name='duracion')
duracion.columns = ['estacion','duracion']
data['retiro_05-10'] = data['Hora_Retiro'].dt.time.between(pd.to_datetime('05:00:00').time(), pd.to_datetime('10:00:00').time())
data['retiro_10-15'] = data['Hora_Retiro'].dt.time.between(pd.to_datetime('10:00:01').time(), pd.to_datetime('15:00:00').time())
data['retiro_15-20'] = data['Hora_Retiro'].dt.time.between(pd.to_datetime('15:00:01').time(), pd.to_datetime('20:00:00').time())
data['retiro_20-01'] = (
    (data['Hora_Retiro'].dt.time >= pd.to_datetime('20:00:01').time()) |
    (data['Hora_Retiro'].dt.time <= pd.to_datetime('01:00:00').time())
)
data['arribo_01-05'] = data['Hora_Arribo'].dt.time.between(pd.to_datetime('01:00:01').time(), pd.to_datetime('04:59:59').time())
data['arribo_05-10'] = data['Hora_Arribo'].dt.time.between(pd.to_datetime('05:00:00').time(), pd.to_datetime('10:00:00').time())
data['arribo_10-15'] = data['Hora_Arribo'].dt.time.between(pd.to_datetime('10:00:01').time(), pd.to_datetime('15:00:00').time())
data['arribo_15-20'] = data['Hora_Arribo'].dt.time.between(pd.to_datetime('15:00:01').time(), pd.to_datetime('20:00:00').time())
data['arribo_20-01'] = (
    (data['Hora_Arribo'].dt.time >= pd.to_datetime('20:00:01').time()) |
    (data['Hora_Arribo'].dt.time <= pd.to_datetime('01:00:00').time())
)
intervalos_retiro = ['retiro_05-10','retiro_10-15','retiro_15-20','retiro_20-01']
intervalos_arribo = ['arribo_05-10','arribo_10-15','arribo_15-20','arribo_20-01','arribo_01-05']
retiros = data.groupby(['Ciclo_Estacion_Retiro'])[intervalos_retiro].sum().reset_index()
retiros.rename(columns={retiros.columns[0]: 'estacion'}, inplace=True)
arribos = data.groupby(['Ciclo_EstacionArribo'])[intervalos_arribo].sum().reset_index()
arribos.rename(columns={arribos.columns[0]: 'estacion'}, inplace=True)

In [ ]:
#################### Inciso 3: Perfiles de uso
estaciones = pd.merge(estacion_a, estacion, left_on='Ciclo_EstacionArribo', right_on='Ciclo_Estacion_Retiro', how='left')
estaciones = estaciones[estaciones['Ciclo_EstacionArribo'] != 'tag 2']
estaciones = estaciones.drop(['Ciclo_Estacion_Retiro'], axis=1)
estaciones.columns = ['estacion','arribos_totales','retiros_totales']

estaciones = pd.merge(estaciones, edad, on='estacion', how='left')
estaciones = pd.merge(estaciones, duracion, on='estacion', how='left')
estaciones = pd.merge(estaciones, retiros, on='estacion', how='left')
estaciones = pd.merge(estaciones, arribos, on='estacion', how='left')
estaciones = estaciones.drop(['arribos_totales','retiros_totales'], axis=1)
estaciones

In [91]:
#### Formar clusters
# Separar la estacion
estacion_id = estaciones['estacion']
estacion_numericos = estaciones.drop(columns=['estacion'])
# Estandarizar los datos
scaler = StandardScaler()
estaciones_scaled = scaler.fit_transform(estacion_numericos)

In [ ]:
# Funcion para determinar numero de clusters
def analisis_k(data, max_k=10):
    distortions = []
    silhouette_scores = []

    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(data)
        distortions.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(data, kmeans.labels_))

    # Gráfico del método del codo
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(range(2, max_k + 1), distortions, marker='o')
    plt.title('Método del codo')
    plt.xlabel('Número de clusters (k)')
    plt.ylabel('Distorsión')

    # Gráfico de la puntuación Silhouette
    plt.subplot(1, 2, 2)
    plt.plot(range(2, max_k + 1), silhouette_scores, marker='o')
    plt.title('Puntuación Silhouette')
    plt.xlabel('Número de clusters (k)')
    plt.ylabel('Silhouette')

    plt.tight_layout()
    # plt.savefig('./Ejercicio_1/figures/codo_silhouette.png')
    plt.show()

    return distortions, silhouette_scores

distortions, silhouette_scores = analisis_k(estaciones_scaled)

In [ ]:
# Implementar K-Means con el número óptimo de clusters
kmeans_model = KMeans(n_clusters=4, random_state=42)
kmeans_labels = kmeans_model.fit_predict(estaciones_scaled)
estaciones['kmeans_labels'] = kmeans_labels
estaciones

In [ ]:
# Implementar DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5, n_jobs=-1)  
dbscan_labels = dbscan.fit_predict(estaciones_scaled)
# Asignar etiquetas dbscan
estaciones['dbsan_labels'] = dbscan_labels
estaciones


In [ ]:
# Kmeans summary
estaciones_temp_kmeans = estaciones.drop(['estacion','dbsan_labels'], axis=1)
kmeans_summary = estaciones_temp_kmeans.groupby('kmeans_labels').agg(['mean']).reset_index()
kmeans_count = estaciones_temp_kmeans.groupby('kmeans_labels')['kmeans_labels'].count().reset_index(name='conteo')
print(kmeans_summary)
print(kmeans_count)

In [ ]:
# dbscan summary
estaciones_temp_dbscan = estaciones.drop(['estacion','kmeans_labels'], axis=1)
db_summary = estaciones_temp_dbscan.groupby('dbsan_labels').agg(['mean']).reset_index()
db_count = estaciones_temp_dbscan.groupby('dbsan_labels')['dbsan_labels'].count().reset_index(name='conteo')

print(db_count.head(10))
print(db_summary)